In [1]:
from __future__ import print_function
from __future__ import division
from gensim.models import Doc2Vec, Word2Vec
import lib.text_extraction as te
import lib.embedding as em
import lib.words as wd
import pandas as pd
import numpy as np
import codecs
import json
import os
import re

Using TensorFlow backend.


In [2]:
test_dir = '../extraction/esempi_descrizioni/'

def list_dir(d):
    return [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

txts = [te.extract_text(f, do_ocr=True) for f in list_dir(test_dir)]


../extraction/esempi_descrizioni/5115873790001.pdf Estrazione testo con Vision API.
../extraction/esempi_descrizioni/02628680346_2081942240001.pdf Estrazione testo con Vision API.


In [3]:
for i, txt in enumerate(txts):
    print('Documento ', i)
    print(txt[:1000])

Documento  0
allegato "A" rep. n.28012 e racc. n.13815
STATUTO
Articolo 1 - DENOMINAZIONE
E' costituita una società a responsabilità limitata denominata "TASI S.R.L.".
Articolo 2 - SEDE
La società ha sede a Tregango (VR).
L'organo amministrativo ha facoltà di istituire e di sopprimere unità locali operative (filiali,
__
uffici senza rappresentanza e agenzie), ovvero di trasferire la sede sociale nell'ambito del
__
Comune sopra indicato, spettando invece ai soci decidere il trasferimento della sede in un
_
Comune diverso da quello sopra indicato e l'istituzione di sedi secondarie.
Articolo 3 - DURATA
La durata della società è fissata fino al 31 dicembre 2050 (trentuno dicembre duemilacin_
quanta).
Articolo 4 - OGGETTO SOCIALE
La società ha per oggetto le seguenti attività:

- il commercio all’ingrosso ed al dettaglio di generi alimentari e bevande di qualsiasi
__
tipo, alcoliche e superalcoliche prodotti da terzi e di produzione propria, sia in Italia
___
che all’estero, esercitato medi

## Embeddings test

In [4]:
txts_tokenized = [wd.tokenize_doc(txt) for txt in txts]

In [5]:
reduced_dictionary_filename = 'first_5000_words.json'
with open(reduced_dictionary_filename) as f:
    reduced_dictionary = set(json.load(f))

gensim_model_filename = 'models/gensim_model_5000.d2v'
gensim_model = Doc2Vec.load(gensim_model_filename)

In [7]:
embeddings = [em.embed_document(gensim_model, txt, reduced_dictionary) for txt in txts_tokenized]

In [8]:
for i, s in enumerate(txts_tokenized[-1]):
    print(i, s)

0 [u'repertorio', u'n', u'NUM', u'raccolta', u'ALPHANUM', u'atto', u'costitutivo', u'della', u'societa', u'a', u'responsabilita', u'limitata', u'tasi', u'srl', u'repubblica', u'italiana', u'il', u'giorno', u'ventinove', u'giugno', u'duemiladiciassette', u'NUM', u'san', u'martino', u'buon', u'albergo', u'studio', u'avanti', u'me', u'paolo', u'carbone', u'nota', u'san', u'martino', u'buon', u'albergo', u'iscritto', u'ruolo', u'distretto', u'notarile', u'verona', u'studio', u'via', u'xx', u'settembre', u'n', u'NUM', u'sono', u'presenti', u'cristofoli', u'mattia', u'nato', u'isola', u'scala', u'vr', u'NUM', u'gennaio', u'NUM', u'domi', u'cilio', u'verona', u'vr', u'via', u'daniele', u'manin', u'ALPHANUM', u'codice', u'fiscale', u'crs', u'mtt', u'ALPHANUM', u'ALPHANUM', u'dichiara', u'essere', u'cittadino', u'italiano']
1 [u'fasoli', u'matteo', u'gino', u'nato', u'tregnago', u'vr', u'NUM', u'giugno', u'NUM', u'domicilio', u'colognola', u'colli', u'vr', u'via', u'giuseppe', u'garibaldi', u'A

In [10]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [11]:
cosine_similarity(embeddings[-1][20],embeddings[-1][-1])

0.88562745

In [12]:
sentences_tokenized = [sent for doc in txts_tokenized for sent in doc] #i should reduce the dictionary also

In [7]:
w2v = Word2Vec(sentences_tokenized, size=100, window=5, min_count=5, workers=4)

In [9]:
w2v.wv['costitutivo']

array([ 0.04124733,  0.08975237,  0.08139414, -0.00763956, -0.10262189,
       -0.08363418,  0.02621282, -0.02461675, -0.00013537, -0.00293726,
       -0.03703222, -0.05810659,  0.01493859,  0.01016551,  0.0539973 ,
       -0.01409576,  0.01948621, -0.06942897,  0.00145047, -0.02296383,
       -0.07333327,  0.03067494, -0.00414452,  0.06249988,  0.02402488,
       -0.10738423,  0.02579944,  0.09187678,  0.06430472,  0.06797387,
       -0.01218945,  0.06671803, -0.00829911, -0.06856858,  0.00110174,
       -0.09487006, -0.03876386,  0.03852043, -0.07464571,  0.03006477,
       -0.01016112, -0.03244141, -0.03297479,  0.08514411, -0.07174642,
        0.00320677, -0.01521917, -0.01043141, -0.07696869,  0.04723798,
        0.01439823, -0.12143837, -0.00936093,  0.06136284,  0.01178922,
       -0.01409232,  0.05657084, -0.07793286,  0.06133021,  0.05784473,
        0.02439136, -0.05320191, -0.01422211,  0.00620693, -0.02311906,
       -0.02699317, -0.03910272, -0.08002615, -0.05018058,  0.04

In [11]:
cosine_similarity(w2v.wv['atto'], w2v.wv['costitutivo'])

0.99804974

In [16]:
w2v.most_similar('poteri')

[(u'soci', 0.9996527433395386),
 (u'societ\xe0', 0.9996300339698792),
 (u'NUM', 0.9996135234832764),
 (u'essere', 0.9996057748794556),
 (u'sociale', 0.9995906949043274),
 (u'ALPHANUM', 0.9995813965797424),
 (u'assemblea', 0.9995689392089844),
 (u'notaio', 0.9995406270027161),
 (u'imprese', 0.9995273947715759),
 (u'mediante', 0.9995262026786804)]

In [21]:
w2v.most_similar(positive=['atto','costitutivo'])

[(u'soci', 0.9995536208152771),
 (u'ALPHANUM', 0.9995061755180359),
 (u'sociale', 0.9994834065437317),
 (u'societ\xe0', 0.9994807243347168),
 (u'NUM', 0.9994748830795288),
 (u'socio', 0.9994532465934753),
 (u'pu\xf2', 0.999414324760437),
 (u'notaio', 0.9994019269943237),
 (u'mediante', 0.9994017481803894),
 (u'la', 0.9993895292282104)]

### Sentencing test

In [13]:
sentenced_txts = [wd.sentences_doc(txt) for txt in txts]

In [31]:
def sentence_label_csv_empty(filenames, sentenced_txts, csv_out):
    csv_out.write(u'filename,sent_index,sentence,label\n')
    for f, sents in zip(filenames, sentenced_txts):
        for i, sent in enumerate(sents):
            csv_out.write(u'{},{},{},not_relevant\n'.format(f,i,sent.replace(u',',u'').replace(u'\n',u' ')))
    

In [32]:
csv_filename = '../extraction/sentence_labels.csv'
with codecs.open(csv_filename, 'w', encoding='utf-8') as csv_out:
    sentence_label_csv_empty(os.listdir(test_dir), sentenced_txts, csv_out)

In [33]:
df = pd.read_csv(csv_filename, sep=',', encoding='utf-8')

In [35]:
df

,filename,sent_index,sentence,label
0,5114243810002-2.pdf,0,"allegato ""A"" rep n28012 e racc n13815 STATUTO ...",not_relevant
1,5114243810002-2.pdf,1,Articolo 2 - SEDE La società ha sede a Tregan...,not_relevant
2,5114243810002-2.pdf,2,L'organo amministrativo ha facoltà di istitui...,not_relevant
3,5114243810002-2.pdf,3,Articolo 3 - DURATA La durata della società è...,not_relevant
4,5114243810002-2.pdf,4,Articolo 4 - OGGETTO SOCIALE La società ha pe...,not_relevant
5,5114243810002-2.pdf,5,- la gestione l’acquisto l’avviamento la vend...,not_relevant
6,5114243810002-2.pdf,6,- la produzione la distribuzione di pasti per...,not_relevant
7,5114243810002-2.pdf,7,- i servizi di consulenza commerciale di mark...,not_relevant
8,5114243810002-2.pdf,8,- l’assunzione di mandati di agenzia e rappre...,not_relevant
9,5114243810002-2.pdf,9,Per il raggiungimento dell'oggetto sociale la...,not_relevant
